In [ ]:
# import first or else jupyter crashes

from desc.plotting import plot_section
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
)
import desc
import matplotlib.pyplot as plt
from desc.basis import ChebyshevFourierSeries


from desc.transform import Transform
from desc.equilibrium import Equilibrium
from desc.basis import ChebyshevZernikeBasis, chebyshev_z
from desc.compute import compute
from desc.grid import LinearGrid, ConcentricGrid, QuadratureGrid, Grid
from desc.compute.utils import get_transforms
from desc.objectives import (
    FixEndCapLambda,
    FixEndCapR,
    FixEndCapZ,
    FixBoundaryR,
    FixBoundaryZ,
    FixPsi,
    FixPressure,
    FixIota,
    ForceBalance,
    ObjectiveFunction,
    CurrentDensity,
    MatchEndCapR,
    MatchEndCapZ,
    MatchEndCapLambda,
    # MidplaneSymmetryOnAxis,
    ForceBalanceAnisotropic,
    MagneticWell,
    FixAnisotropy,
    # EndCapMatchLambda_Rot,
    # EndCapMatchR_Rot,
    # EndCapMatchZ_Rot,
    FixAnisotropy,
)
import numpy as np
from desc.optimize import Optimizer
from desc.profiles import PowerSeriesProfile
from desc.geometry import FourierRZToroidalSurface
from scipy.constants import mu_0
from mayavi import mlab


def chebygrid(N_grid):
    return np.concatenate(
        (
            [0],
            (-np.cos((2 * np.arange(N_grid) + 1) * np.pi / (2 * N_grid)) + 1) * np.pi,
            [2 * np.pi],
        )
    )


def grid_gen(L_grid, M_grid, N_grid, node_pattern="jacobi"):
    LMnodes = ConcentricGrid(L=L_grid, M=M_grid, N=0, node_pattern=node_pattern).nodes[
        :, :2
    ]
    Nnodes = chebygrid(N_grid)
    lm = np.tile(LMnodes, (Nnodes.size, 1))
    n = np.tile(Nnodes.reshape(-1, 1), (1, LMnodes.shape[0])).reshape(-1, 1)
    nodes = np.concatenate((lm, n), axis=1)

    # RG: weights and spacing defined here
    # just for the sake of compilation. Must be checked
    weights = np.ones(nodes.shape[0])
    spacing = np.ones_like(nodes)

    spacing[1:, 1] = np.diff(nodes[:, 1])
    spacing[1:, 2] = np.diff(nodes[:, 2])

    return Grid(nodes, spacing=spacing, weights=weights)


def get_lm_mode(basis, coeff, zeta, L, M, func_zeta=chebyshev_z):
    modes = basis.modes
    lm = 0
    for i, (l, m, n) in enumerate(modes):
        if l == L and m == M:
            lm += func_zeta(zeta, n) * coeff[i]
    return lm


plt.rcParams.update(
    {
        "font.size": 16,  # Default text size
        "axes.titlesize": 18,  # Title font
        "axes.labelsize": 16,  # x/y label font
        "xtick.labelsize": 14,  # Tick font
        "ytick.labelsize": 14,
        "legend.fontsize": 14,
    }
)


## test
surf = FourierRZToroidalSurface(
    R_lmn=[10, 1, -0.5, 0.5],
    modes_R=[[0, 0], [1, 0], [1, 2], [1, 4]],
    Z_lmn=[0, -1, 0.5, -0.5],
    modes_Z=[[0, 0], [-1, 0], [-1, 2], [-1, 4]],
    NFP=1,
    sym=False,
    mirror=True,
)

## test
surf = FourierRZToroidalSurface(
    R_lmn=[10, 1, -0.5],
    modes_R=[[0, 0], [1, 0], [1, 2]],
    Z_lmn=[0, -1, 0.5],
    modes_Z=[[0, 0], [-1, 0], [-1, 2]],
    NFP=1,
    sym=False,
    mirror=True,
)


p = PowerSeriesProfile(params=[0.001 / mu_0, -0.001 / mu_0], modes=[0, 2])
iota = PowerSeriesProfile(params=[0.0, 0])


eq = Equilibrium(
    surface=surf,
    L=5,
    M=2,
    N=8,
    mirror=True,
    pressure=p,
    iota=iota,
    sym=False,
)

In [ ]:
p.set_params(0, a=0e6)  # = array([-1000])
p.set_params(2, a=-0e6)  # = array([-1000])

In [ ]:
#### OPTIMIZE FROM SFLM GRID #####

In [ ]:
### get sflm grid
### Paramaterized surface defined by theta and z


# Define the parametric functions for the ellipse
def get_x(theta, z, rho, c):
    return rho * (1 + z / c) * np.cos(theta)


def get_y(theta, z, rho, c):
    return rho * (1 - z / c) * np.sin(theta)


def get_z(theta, z):
    return z

In [ ]:
### INSTEAD of a Fieldine trace, get a rho surface
N_fieldlines = 100
res_fieldlines = 100

z = np.linspace(-np.pi, np.pi, res_fieldlines)
angle = np.linspace(0, 2 * np.pi, N_fieldlines, endpoint=False)
R_fieldlines = [0.1]
B0 = 0.25
c = 3.5
alpha_x_mesh = []
alpha_y_mesh = []
alpha_z_mesh = []
alpha_theta_mesh = []

for R in R_fieldlines:

    for theta in angle:
        x = get_x(theta, z, R, c)
        y = get_y(theta, z, R, c)
        theta_ = np.ones_like(x) * theta

        alpha_x_mesh += [x]
        alpha_y_mesh += [y]
        alpha_z_mesh += [z]
        alpha_theta_mesh += [theta_]

# get gradrho
X = np.ravel(alpha_x_mesh)
Z = np.ravel(alpha_z_mesh)
Y = np.ravel(alpha_y_mesh)
R = np.sqrt(X**2 + Y**2)
Theta = np.arctan(Y, X)
Theta = np.mod(Theta, 2 * np.pi)

In [ ]:
plt.figure()
plt.plot(np.array(alpha_x_mesh)[:, 0], np.array(alpha_y_mesh)[:, 0], ".")
plt.axis("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.title(
    "Max Aspect Ratio: {:.2f}".format(
        np.max(np.array(alpha_y_mesh)[:, 0]) / np.max(np.array(alpha_x_mesh)[:, 0])
    )
)

In [ ]:
#####

In [ ]:
P = np.array([alpha_x_mesh, alpha_y_mesh, alpha_z_mesh]).T

In [ ]:
plt.figure()
plt.plot(alpha_z_mesh, alpha_x_mesh, ".")
plt.grid()

In [ ]:
plt.figure()
plt.plot(alpha_y_mesh, alpha_x_mesh, ".")
plt.grid()

In [ ]:
xf = np.ravel(P[:, :, 0])
yf = np.ravel(P[:, :, 1])
zf = np.ravel(P[:, :, 2])


### transform z to zeta
z_min = np.min(zf)
z_max = np.max(zf)

# Shift and scale to get zeta in [0, 2π]
zeta = 2 * np.pi * (zf - z_min) / (z_max - z_min)
# theta is
theta_vals = np.arctan2(yf, xf)
theta = np.mod(theta_vals, 2 * np.pi)  # Ensure values in [0, 2π]

In [ ]:
## test
surf = FourierRZToroidalSurface(
    R_lmn=[10, 1, -0.5],
    modes_R=[[0, 0], [1, 0], [1, 2]],
    Z_lmn=[0, -1, 0.5],
    modes_Z=[[0, 0], [-1, 0], [-1, 2]],
    NFP=1,
    sym=False,
    mirror=True,
)

In [ ]:
grid = grid_gen(4, 10, 40)
data = surf.compute(["X", "Y", "Z"], grid=grid)

# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]

In [ ]:
x.shape

In [ ]:
plt.figure()
plt.plot(y, x, ".")

In [ ]:
plt.figure()
plt.plot(y, z, ".")
plt.plot(y[0:20], z[0:20], ".")
plt.plot(y[-20:-1], z[-20:-1], ".")

In [ ]:
np.max(z[0:20]) / np.max(x[0:20] - np.average(x[0:20])), np.max(
    x[-20:-1] - np.average(x[-20:-1])
) / np.max(z[-20:-1])

In [ ]:
## run optimization

from scipy.spatial import cKDTree
import numpy as np

In [ ]:
### transform points to get a target

xf = np.ravel(P[:, :, 0])
yf = np.ravel(P[:, :, 1])
zf = np.ravel(P[:, :, 2])


### transform z to zeta
z_min = np.min(zf)
z_max = np.max(zf)

### old length
L_old = np.abs(z_min) + z_max
L_new = 2 * np.pi
scale = L_new / L_old

# Shift and scale to get zeta in [0, 2π]
Y = 2 * np.pi * (zf - z_min) / (z_max - z_min)
X = xf * scale + 10
Z = yf * scale

In [ ]:
####

In [ ]:
plt.figure()
plt.plot(Y, Z, ".")

In [ ]:
plt.plot(Y, X, ".")

In [ ]:
tree = cKDTree(np.array([X, Y, Z]).T)
grid = grid_gen(1, 100, 60)

In [ ]:
# Define a penalty function for points outside the field line boundary
def penalty_function(dofs):

    surf.R_lmn = dofs[0 : int(len(dofs) / 2)]

    surf.Z_lmn = dofs[int(len(dofs) / 2) : :]

    data = surf.compute(["X", "Y", "Z"], grid=grid)

    xf = data["X"]
    yf = data["Y"]
    zf = data["Z"]

    fourier_points = np.array([xf, yf, zf]).T

    # Calculate the distance from each Fourier point to the nearest field line point
    distances, _ = tree.query(fourier_points)

    # Penalize points with distances above a threshold
    # threshold_distance = 0  # Adjust this threshold based on your geometry
    # penalties = np.minimize(distances - threshold_distance, 0)  # Only penalize outside points
    return np.sum(distances**2)  # Sum of squared penalties

In [ ]:
surf.change_resolution(1, 1)

In [ ]:
dofs = np.ravel(np.array([surf.R_lmn, surf.Z_lmn]))

In [ ]:
dofs[0 : int(len(dofs) / 2)]

In [ ]:
test = penalty_function(dofs)

In [ ]:
test

In [ ]:
from scipy.optimize import minimize

In [ ]:
res = minimize(
    penalty_function,
    dofs,
    jac=False,
    method="L-BFGS-B",
    options={"disp": True, "maxiter": 100},
)

In [ ]:
dofs = res.x

In [ ]:
surf.R_lmn = dofs[0 : int(len(dofs) / 2)]

surf.Z_lmn = dofs[int(len(dofs) / 2) : :]

In [ ]:
grid = grid_gen(4, 10, 40)
data = surf.compute(["X", "Y", "Z"], grid=grid)

# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]

In [ ]:
plt.figure()
plt.plot(y, x, ".")

In [ ]:
plt.figure()
plt.plot(y, z, ".")
plt.plot(y[0:50], z[0:50], ".")
plt.plot(y[-50:-1], z[-50:-1], ".")

In [ ]:
np.max(z[0:20]) / np.max(x[0:20] - np.average(x[0:20])), np.max(
    x[-20:-1] - np.average(x[-20:-1])
) / np.max(z[-20:-1])

In [ ]:
# surf.save('SFLM_mediumMR_5p2.hdf5')

In [ ]:
# surf = FourierRZToroidalSurface.load('SFLM_highMR_11.hdf5')

In [ ]:
surf = FourierRZToroidalSurface.load("SFLM_lowMR_2p6.hdf5")

In [ ]:
surf_init = surf.copy()

In [ ]:
eq = Equilibrium(
    surface=surf_init,
    L=10,
    M=2,
    N=12,
    mirror=True,
    pressure=p,
    iota=iota,
    sym=False,
)

In [ ]:
eq_start = eq.copy()

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 64)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p", "beta_a", "Y"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p"]

plt.figure()
plt.plot(zeta_axis, B_axis)
plt.title("MR = {}".format(np.max(B_axis) / np.min(B_axis)))

In [ ]:
### Lower B magnitude by 10x

eq.Psi = 1 / np.max(B_axis)

In [ ]:
eq.Psi

In [ ]:
# eq.change_resolution(L = 10, M = 4, N = 8)

In [ ]:
constraints = (
    # FixEndCapLambda(0, eq=eq),
    # FixEndCapR(0, eq=eq),
    # FixEndCapZ(0, eq=eq),
    # FixEndCapLambda(2*np.pi, eq=eq),
    # FixEndCapR(2*np.pi, eq=eq),
    # FixEndCapZ(2*np.pi, eq=eq),
    # MatchEndCapR(eq=eq),
    # MatchEndCapZ(eq=eq),
    # MatchEndCapLambda,
    # EndCapMatchLambda_Rot(eq=eq),
    # EndCapMatchR_Rot(eq=eq),
    # EndCapMatchZ_Rot(eq=eq),
    FixBoundaryR(eq=eq),
    FixBoundaryZ(eq=eq),
    FixPressure(eq=eq),
    FixIota(eq=eq),
    FixPsi(eq=eq),
    # FixAnisotropy(eq=eq),
)
optimizer = Optimizer("lsq-exact")
grid = grid_gen(20, 20, 20)
# objectives = CurrentDensity(eq=eq, grid=grid)
# objectives = ForceBalance(eq=eq, grid=grid)
# objectives = [ForceBalance(eq=eq, grid=grid), 100.0 * symmetry_obj]
# obj = ObjectiveFunction(objectives=objectives)
# force_obj = ForceBalanceAnisotropic(eq=eq, grid=grid)
# obj = ObjectiveFunction(objectives=[force_obj])

aFB = ForceBalance(eq=eq, grid=grid, weight=1e8)

stab = MagneticWell(eq=eq)
obj = ObjectiveFunction(objectives=[aFB])

eq.solve(
    objective=obj,
    constraints=constraints,
    optimizer=optimizer,
    ftol=1e-8,
    xtol=1e-16,
    gtol=1e-16,
    maxiter=50,
    verbose=2,
)

In [ ]:
f = (
    eq.compute("<|F|>_vol")["<|F|>_vol"]
    / eq.compute("<|grad(|B|^2)|/2mu0>_vol")["<|grad(|B|^2)|/2mu0>_vol"]
)

print(f"Force error after eq.solve(): {f:.4e}")

In [ ]:
eq.compute("|F|", grid=grid)["|F|"].max()

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 64)
rho_axis = np.ones_like(zeta_axis) * 0
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p", "Y"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p"]

plt.figure()
plt.plot(zeta_axis, B_axis, label=r"$|B|$")
plt.title("MR = {:.2f}".format(np.max(B_axis) / np.min(B_axis)))
plt.legend()

In [ ]:
from desc.plotting import *

In [ ]:
plot_section(eq, "|F|", norm_F=True, log=True)

In [ ]:
# %% Get fieldlines on the LCFS
# assert isinstance(eq, Equilibrium)

Nz = 200
Nt = 100
rho_surf = 0.5

theta = np.linspace(0, 2 * np.pi, Nt)
zeta = np.linspace(0, 2 * np.pi, Nz, endpoint=True)

theta_mesh, zeta_mesh = np.meshgrid(theta, zeta, indexing="ij")
shape = theta_mesh.shape
coords_sfl = np.stack(
    [
        np.ones_like(theta_mesh.reshape(-1)) * rho_surf,
        theta_mesh.reshape(-1),
        zeta_mesh.reshape(-1),
    ],
    axis=-1,
)

coords_geo = eq.compute_theta_coords(coords_sfl)
coords_data = eq.compute(
    ["R", "Z", "zeta", "p", "|B|", "kappa"], grid=Grid(nodes=coords_geo, sort=False)
)


lines = np.array([coords_data["R"], coords_data["zeta"], coords_data["Z"]])
lines = lines.reshape(3, shape[0], shape[1])
lines = np.moveaxis(lines, [0, 1], [1, 0])

B_on_lines = coords_data["|B|"]
B_on_lines = B_on_lines.reshape(shape[0], shape[1])


p_on_lines = coords_data["p"]
p_on_lines = p_on_lines.reshape(shape[0], shape[1])
k_on_lines = coords_data["kappa"]
k_on_lines = np.linalg.norm(k_on_lines, axis=-1)
k_on_lines = k_on_lines.reshape(shape[0], shape[1])

In [ ]:
data_mesh = eq.compute(
    [
        "theta",
        "X",
        "Y",
        "Z",
        "|B|",
        "B",
        "p",
        "J",
        "e^theta",
        "e^rho",
        "e^zeta",
        "grad(alpha)",
    ],
    grid=Grid(nodes=coords_geo, sort=False),
)

### axial pressure profile
mu0 = 4 * np.pi * 1e-7

# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_axis = data_axis["p"]


# ---- Grid for midplane slice (zeta = pi, theta = 0, rho from 0 to 1) ----
rho_mid = np.linspace(0, 1, 100)
theta_mid = np.zeros_like(rho_mid)
zeta_mid = np.ones_like(rho_mid) * np.pi
grid_mid = Grid(np.stack([rho_mid, theta_mid, zeta_mid], axis=1))

# Compute on midplane profile
data_mid = eq.compute(["p", "|B|", "p"], grid=grid_mid)
B_mid = data_mid["|B|"]
p_mid = data_mid["p"]


X2d = np.reshape(data_mesh["X"], (Nz, Nt))
Y2d = np.reshape(data_mesh["Y"], (Nz, Nt))
Z2d = np.reshape(data_mesh["Z"], (Nz, Nt))
B2d = np.reshape(data_mesh["|B|"], (Nz, Nt))
J2d = np.reshape(data_mesh["J"], (Nz, Nt, 3))
theta2d = np.reshape(data_mesh["theta"], (Nz, Nt))

etheta = np.reshape(data_mesh["e^theta"], (Nz, Nt, 3))
ezeta = np.reshape(data_mesh["e^zeta"], (Nz, Nt, 3))
erho = np.reshape(data_mesh["e^rho"], (Nz, Nt, 3))

e_theta_n = etheta / np.linalg.norm(etheta, axis=-1)[:, :, None]
e_zeta_n = ezeta / np.linalg.norm(ezeta, axis=-1)[:, :, None]
e_rho_n = erho / np.linalg.norm(erho, axis=-1)[:, :, None]

J2d_n = J2d / np.linalg.norm(J2d, axis=-1)[:, :, None]

comp_theta = np.sum(J2d_n * e_theta_n, axis=-1)
comp_zeta = np.sum(J2d_n * e_zeta_n, axis=-1)
comp_rho = np.sum(J2d_n * e_rho_n, axis=-1)

Bv = np.reshape(data_mesh["B"], (Nz, Nt, 3))
B2d_n = Bv / np.linalg.norm(Bv, axis=-1)[:, :, None]

JdotB = np.sum(J2d_n * B2d_n, axis=-1)

# gradp = np.reshape(data_mesh['grad(p)'],(Nz,Nt,3))
# gradp_n =  gradp / np.linalg.norm(gradp,axis = -1)[:,:,None]

# Jdotgradp = np.sum(J2d_n*gradp_n,axis = -1)

# Bdotgradp = np.sum(B2d_n*gradp_n,axis = -1)

JdotB2d_n = np.sum(J2d_n * B2d_n, axis=-1)

gradalpha = np.reshape(data_mesh["grad(alpha)"], (Nz, Nt, 3))
gradalpha_n = gradalpha / np.linalg.norm(gradalpha, axis=-1)[:, :, None]

Jdotgradalpha_n = np.sum(J2d_n * gradalpha_n, axis=-1)

beta = 2 * (data_mesh["p"]) * mu_0 / (data_mesh["|B|"] ** 2 + 1e-20)
beta_axis = 2 * (data_axis["p"]) * mu_0 / (data_axis["|B|"] ** 2 + 1e-20)
beta_midplane = 2 * (data_mid["p"]) * mu_0 / (data_mid["|B|"] ** 2 + 1e-20)


beta2d = np.reshape(beta, (Nz, Nt))

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(16, 11))

nlevels = 500
cmap = "coolwarm"

# field lines
i = 0
for line in lines[::10]:
    fieldlines = axs[0, 0].scatter(line[1], line[2], c=B_on_lines[i], s=0.2)
    i = i + 10
fig.colorbar(fieldlines, ax=axs[0, 0], label=r"|B|")

#

axs[0, 1].plot(zeta_axis, p_axis, label=r"$p$")
axs[0, 1].legend()

axs[0, 2].plot(rho_mid, p_mid, label=r"$p$")
axs[0, 2].legend()


im = axs[1, 0].contour(Y2d, theta2d, beta2d, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[1, 0])


im = axs[1, 1].plot(zeta_axis, beta_axis)
im = axs[1, 2].plot(rho_mid, beta_midplane)


# J
im = axs[2, 1].contour(Y2d, theta2d, comp_rho, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 1])

im = axs[2, 0].contour(Y2d, theta2d, Jdotgradalpha_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 0])

im = axs[2, 2].contour(Y2d, theta2d, JdotB2d_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 2])


# titles
axs[0, 0].set_title(r"$\rho = {}$ Field lines".format(rho_surf))
axs[0, 1].set_title("axial profile")
axs[0, 2].set_title("radial profile (midplane)")

axs[1, 0].set_title(r"$\beta$ at $\rho = {}$ ".format(rho_surf))
axs[1, 1].set_title(r"axial $\beta$")
axs[1, 2].set_title(r"radial $\beta$")

axs[2, 2].set_title(r"$J \cdot b$")
axs[2, 0].set_title(r"$J \cdot \nabla \alpha$")
axs[2, 1].set_title(r"$J \cdot e^\rho$")

axs[0, 0].set_ylabel(r"$Z$")
axs[0, 1].set_ylabel(r"$pressure(Pa)$")
axs[0, 2].set_ylabel(r"$pressure(Pa)$")

axs[2, 2].set_ylabel(r"$\theta$")
axs[2, 0].set_ylabel(r"$\theta$")
axs[2, 1].set_ylabel(r"$\theta$")


axs[2, 0].set_xlabel(r"$Y$")
axs[2, 1].set_xlabel(r"$Y$")
axs[2, 2].set_xlabel(r"$Y$")


axs[0, 0].grid()
axs[1, 1].grid()
axs[1, 2].grid()

axs[2, 2].grid()
axs[2, 0].grid()
axs[2, 1].grid()

fig.tight_layout()

In [ ]:
# Create a figure and axis manually
fig, ax = plt.subplots(figsize=(12, 8))

# Plot on the custom axis
desc.plotting.plot_1d(eq, name="magnetic well", ax=ax, lw=2)

In [ ]:
# %% Get fieldlines on the LCFS
# assert isinstance(eq, Equilibrium)

Nz = 50
Nt = 50
Nr = 1

theta = np.linspace(0, 2 * np.pi, Nt)
zeta = np.linspace(0, 2 * np.pi, Nz, endpoint=False)
rho = np.linspace(1, 1, Nr)

full_coords = []
for r in rho:

    theta_mesh, zeta_mesh = np.meshgrid(theta, zeta, indexing="ij")
    shape = theta_mesh.shape
    coords_sfl = np.stack(
        [
            np.ones_like(theta_mesh.reshape(-1)) * r,
            theta_mesh.reshape(-1),
            zeta_mesh.reshape(-1),
        ],
        axis=-1,
    )

    coords_geo = eq.compute_theta_coords(coords_sfl)

    full_coords.append(coords_geo)

full_coords = np.array(full_coords)

In [ ]:
coords_geo = np.array(
    [
        full_coords[:, :, 0].ravel(),
        full_coords[:, :, 1].ravel(),
        full_coords[:, :, 2].ravel(),
    ]
).T
grid = Grid(nodes=coords_geo, sort=False)
# data = eq.compute(['isodynamicity','X','Y','Z','|B|','<beta>_vol','beta_a','|F|','p','grad(p)','J','|J|','J_rho','J_theta','J_zeta','J*B','J_parallel'], grid = grid)

In [ ]:
data = eq.compute(
    [
        "isodynamicity",
        "X",
        "Y",
        "Z",
        "|B|",
        "beta_a",
        "|F|",
        "p",
        "grad(p)",
        "J",
        "|J|",
        "J_rho",
        "J_theta",
        "J_zeta",
        "J*B",
        "J_parallel",
    ],
    grid=grid,
)

In [ ]:
beta = 2 * ((data["p"]) * mu_0 / (data["|B|"] ** 2 + 1e-20)).reshape(Nt, Nz)
B = data["|B|"].reshape(Nt, Nz)

In [ ]:
# 3D

x = data["X"].reshape(Nt, Nz)
y = data["Y"].reshape(Nt, Nz)
z = data["Z"].reshape(Nt, Nz)

fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.0, 0.0, 0.0))


lines = np.array([x, y, z]).T
lines = np.swapaxes(lines, 0, 1)
for line in lines[::2]:
    mlab.plot3d(line[:, 0], line[:, 1], line[:, 2], tube_radius=0.002)


mlab.mesh(x, y, z, scalars=B, opacity=0.9, colormap="viridis")

mlab.colorbar(title="|B|")


mlab.view(azimuth=105, elevation=75, distance=10, focalpoint="auto")  # Adjust as needed

mlab.show()

In [ ]:
# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]
# b = data['p']
# bv = data['grad(p)']
bv = data["B"]

fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.0, 0.0, 0.0))

points = mlab.points3d(x, y, z, scale_factor=0.01, scale_mode="none")
arrows = mlab.quiver3d(
    x,
    y,
    z,
    bv[:, 0],
    bv[:, 1],
    bv[:, 2],
    scale_factor=0.1,
    line_width=10,
    scale_mode="none",
)

bv = data["J"]
arrows1 = mlab.quiver3d(
    x,
    y,
    z,
    bv[:, 0],
    bv[:, 1],
    bv[:, 2],
    scale_factor=0.1,
    line_width=10,
    scale_mode="none",
    color=(1, 0, 1),
)

# bv = data['e^zeta']
# arrows2 = mlab.quiver3d(x,y,z,bv[:,0],bv[:,1],bv[:,2], scale_factor=.1, line_width=10,scale_mode = 'none',color = (1,0,0))

# bv = data['e^theta']
# arrows3 = mlab.quiver3d(x,y,z,bv[:,0],bv[:,1],bv[:,2], scale_factor=.1, line_width=10,scale_mode = 'none',color = (0,0,1))


# mlab.colorbar(arrows)


mlab.show()

In [ ]:
### Add pressure
eq_no_p = eq.copy()

In [ ]:
eq_8 = eq.copy()

In [ ]:
eq = eq_no_p.copy()

In [ ]:
eq_high = eq.copy()

In [ ]:
eq.pressure.set_params(0, a=5e3)
eq.pressure.set_params(2, a=-5e3)

In [ ]:
eq.pressure.set_params(0, a=3e4)
eq.pressure.set_params(2, a=-3e4)

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 64)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p", "beta_a", "Y"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p"]

plt.figure()
plt.plot(zeta_axis, B_axis)
plt.title("MR = {}".format(np.max(B_axis) / np.min(B_axis)))

In [ ]:
constraints = (
    # FixEndCapLambda(0, eq=eq),
    # FixEndCapR(0, eq=eq),
    # FixEndCapZ(0, eq=eq),
    # FixEndCapLambda(2*np.pi, eq=eq),
    # FixEndCapR(2*np.pi, eq=eq),
    # FixEndCapZ(2*np.pi, eq=eq),
    # MatchEndCapR(eq=eq),
    # MatchEndCapZ(eq=eq),
    # MatchEndCapLambda,
    # EndCapMatchLambda_Rot(eq=eq),
    # EndCapMatchR_Rot(eq=eq),
    # EndCapMatchZ_Rot(eq=eq),
    FixBoundaryR(eq=eq),
    FixBoundaryZ(eq=eq),
    FixPressure(eq=eq),
    FixIota(eq=eq),
    FixPsi(eq=eq),
    # FixAnisotropy(eq=eq),
)
optimizer = Optimizer("lsq-exact")
grid = grid_gen(20, 20, 20)
# objectives = CurrentDensity(eq=eq, grid=grid)
# objectives = ForceBalance(eq=eq, grid=grid)
# objectives = [ForceBalance(eq=eq, grid=grid), 100.0 * symmetry_obj]
# obj = ObjectiveFunction(objectives=objectives)
# force_obj = ForceBalanceAnisotropic(eq=eq, grid=grid)
# obj = ObjectiveFunction(objectives=[force_obj])

aFB = ForceBalance(eq=eq, grid=grid, weight=1e8)

stab = MagneticWell(eq=eq)
obj = ObjectiveFunction(objectives=[aFB])

eq.solve(
    objective=obj,
    constraints=constraints,
    optimizer=optimizer,
    ftol=1e-8,
    xtol=1e-16,
    gtol=1e-16,
    maxiter=25,
    verbose=3,
)

In [ ]:
f = (
    eq.compute("<|F|>_vol")["<|F|>_vol"]
    / eq.compute("<|grad(|B|^2)|/2mu0>_vol")["<|grad(|B|^2)|/2mu0>_vol"]
)

print(f"Force error after eq.solve(): {f:.4e}")

In [ ]:
f

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 64)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p", "beta_a", "Y"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p"]

plt.figure()
plt.plot(zeta_axis, B_axis)
plt.title("MR = {}".format(np.max(B_axis) / np.min(B_axis)))

In [ ]:
# %% Get fieldlines on the LCFS
# assert isinstance(eq, Equilibrium)

Nz = 200
Nt = 100
rho_surf = 1

theta = np.linspace(0, 2 * np.pi, Nt)
zeta = np.linspace(0, 2 * np.pi, Nz, endpoint=True)

theta_mesh, zeta_mesh = np.meshgrid(theta, zeta, indexing="ij")
shape = theta_mesh.shape
coords_sfl = np.stack(
    [
        np.ones_like(theta_mesh.reshape(-1)) * rho_surf,
        theta_mesh.reshape(-1),
        zeta_mesh.reshape(-1),
    ],
    axis=-1,
)

coords_geo = eq.compute_theta_coords(coords_sfl)
coords_data = eq.compute(
    ["R", "Z", "zeta", "p", "|B|", "kappa"], grid=Grid(nodes=coords_geo, sort=False)
)


lines = np.array([coords_data["R"], coords_data["zeta"], coords_data["Z"]])
lines = lines.reshape(3, shape[0], shape[1])
lines = np.moveaxis(lines, [0, 1], [1, 0])

B_on_lines = coords_data["|B|"]
B_on_lines = B_on_lines.reshape(shape[0], shape[1])


p_on_lines = coords_data["p"]
p_on_lines = p_on_lines.reshape(shape[0], shape[1])
k_on_lines = coords_data["kappa"]
k_on_lines = np.linalg.norm(k_on_lines, axis=-1)
k_on_lines = k_on_lines.reshape(shape[0], shape[1])

In [ ]:
p_on_lines

In [ ]:
from scipy.integrate import simpson as simps

# pressure-weighted curvature integrand
integrand = p_on_lines * k_on_lines  # shape (Nt, Nz)

dR = np.gradient(lines[:, 0, :], axis=1)
dZ = np.gradient(lines[:, 2, :], axis=1)
dphi = np.gradient(lines[:, 1, :], axis=1)
R = lines[:, 0, :]

ds = np.sqrt(dR**2 + dZ**2 + (R * dphi) ** 2)  # shape (Nt, Nz)

# integrate for each field line using Simpson's rule
stability_integral = np.array(
    [
        simps(integrand[i] * ds[i], dx=1.0)  # dx=1.0 since spacing is uniform in zeta
        for i in range(Nt)
    ]
)

In [ ]:
stability_integral

In [ ]:
data_mesh = eq.compute(
    [
        "theta",
        "X",
        "Y",
        "Z",
        "|B|",
        "B",
        "p",
        "J",
        "e^theta",
        "e^rho",
        "e^zeta",
        "grad(alpha)",
    ],
    grid=Grid(nodes=coords_geo, sort=False),
)

### axial pressure profile
mu0 = 4 * np.pi * 1e-7

# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_axis = data_axis["p"]


# ---- Grid for midplane slice (zeta = pi, theta = 0, rho from 0 to 1) ----
rho_mid = np.linspace(0, 1, 100)
theta_mid = np.zeros_like(rho_mid)
zeta_mid = np.ones_like(rho_mid) * np.pi
grid_mid = Grid(np.stack([rho_mid, theta_mid, zeta_mid], axis=1))

# Compute on midplane profile
data_mid = eq.compute(["p", "|B|", "p"], grid=grid_mid)
B_mid = data_mid["|B|"]
p_mid = data_mid["p"]


X2d = np.reshape(data_mesh["X"], (Nz, Nt))
Y2d = np.reshape(data_mesh["Y"], (Nz, Nt))
Z2d = np.reshape(data_mesh["Z"], (Nz, Nt))
B2d = np.reshape(data_mesh["|B|"], (Nz, Nt))
J2d = np.reshape(data_mesh["J"], (Nz, Nt, 3))
theta2d = np.reshape(data_mesh["theta"], (Nz, Nt))

etheta = np.reshape(data_mesh["e^theta"], (Nz, Nt, 3))
ezeta = np.reshape(data_mesh["e^zeta"], (Nz, Nt, 3))
erho = np.reshape(data_mesh["e^rho"], (Nz, Nt, 3))

e_theta_n = etheta / np.linalg.norm(etheta, axis=-1)[:, :, None]
e_zeta_n = ezeta / np.linalg.norm(ezeta, axis=-1)[:, :, None]
e_rho_n = erho / np.linalg.norm(erho, axis=-1)[:, :, None]

J2d_n = J2d / np.linalg.norm(J2d, axis=-1)[:, :, None]

comp_theta = np.sum(J2d_n * e_theta_n, axis=-1)
comp_zeta = np.sum(J2d_n * e_zeta_n, axis=-1)
comp_rho = np.sum(J2d_n * e_rho_n, axis=-1)

Bv = np.reshape(data_mesh["B"], (Nz, Nt, 3))
B2d_n = Bv / np.linalg.norm(Bv, axis=-1)[:, :, None]

JdotB = np.sum(J2d_n * B2d_n, axis=-1)

# gradp = np.reshape(data_mesh['grad(p)'],(Nz,Nt,3))
# gradp_n =  gradp / np.linalg.norm(gradp,axis = -1)[:,:,None]

# Jdotgradp = np.sum(J2d_n*gradp_n,axis = -1)

# Bdotgradp = np.sum(B2d_n*gradp_n,axis = -1)

JdotB2d_n = np.sum(J2d_n * B2d_n, axis=-1)

gradalpha = np.reshape(data_mesh["grad(alpha)"], (Nz, Nt, 3))
gradalpha_n = gradalpha / np.linalg.norm(gradalpha, axis=-1)[:, :, None]

Jdotgradalpha_n = np.sum(J2d_n * gradalpha_n, axis=-1)

beta = 2 * (data_mesh["p"]) * mu_0 / (data_mesh["|B|"] ** 2 + 1e-20)
beta_axis = 2 * (data_axis["p"]) * mu_0 / (data_axis["|B|"] ** 2 + 1e-20)
beta_midplane = 2 * (data_mid["p"]) * mu_0 / (data_mid["|B|"] ** 2 + 1e-20)


beta2d = np.reshape(beta, (Nz, Nt))

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(16, 11))

nlevels = 500
cmap = "coolwarm"

# field lines
i = 0
for line in lines[::10]:
    fieldlines = axs[0, 0].scatter(line[1], line[2], c=B_on_lines[i], s=0.2)
    i = i + 10
fig.colorbar(fieldlines, ax=axs[0, 0], label=r"|B|")

#

axs[0, 1].plot(zeta_axis, p_axis, label=r"$p$")
axs[0, 1].legend()

axs[0, 2].plot(rho_mid, p_mid, label=r"$p$")
axs[0, 2].legend()


im = axs[1, 0].contour(Y2d, theta2d, beta2d, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[1, 0])


im = axs[1, 1].plot(zeta_axis, beta_axis)
im = axs[1, 2].plot(rho_mid, beta_midplane)


# J
im = axs[2, 1].contour(Y2d, theta2d, comp_rho, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 1])

im = axs[2, 0].contour(Y2d, theta2d, Jdotgradalpha_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 0])

im = axs[2, 2].contour(Y2d, theta2d, JdotB2d_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 2])


# titles
axs[0, 0].set_title(r"$\rho = {}$ Field lines".format(rho_surf))
axs[0, 1].set_title("axial profile")
axs[0, 2].set_title("radial profile (midplane)")

axs[1, 0].set_title(r"$\beta$ at $\rho = {}$ ".format(rho_surf))
axs[1, 1].set_title(r"axial $\beta$")
axs[1, 2].set_title(r"radial $\beta$")

axs[2, 2].set_title(r"$J \cdot b$")
axs[2, 0].set_title(r"$J \cdot \nabla \alpha$")
axs[2, 1].set_title(r"$J \cdot e^\rho$")

axs[0, 0].set_ylabel(r"$Z$")
axs[0, 1].set_ylabel(r"$pressure(Pa)$")
axs[0, 2].set_ylabel(r"$pressure(Pa)$")

axs[2, 2].set_ylabel(r"$\theta$")
axs[2, 0].set_ylabel(r"$\theta$")
axs[2, 1].set_ylabel(r"$\theta$")


axs[2, 0].set_xlabel(r"$Y$")
axs[2, 1].set_xlabel(r"$Y$")
axs[2, 2].set_xlabel(r"$Y$")


axs[0, 0].grid()
axs[1, 1].grid()
axs[1, 2].grid()

axs[2, 2].grid()
axs[2, 0].grid()
axs[2, 1].grid()

fig.tight_layout()

In [ ]:
plot_section(eq, "|F|", norm_F=True, log=True)

In [ ]:
# Create a figure and axis manually
fig, ax = plt.subplots(figsize=(12, 8))

# Plot on the custom axis
desc.plotting.plot_1d(eq, name="magnetic well", ax=ax, lw=2)

In [ ]:
# Create a figure and axis manually
fig, ax = plt.subplots(figsize=(6, 4))


# Plot on the custom axis
desc.plotting.plot_1d(eq, name="magnetic well", ax=ax, lw=2, linecolor="blue")


plt.grid()

In [ ]:
# Create a figure and axis manually
fig, ax = plt.subplots(figsize=(6, 4))


# Plot on the custom axis
desc.plotting.plot_1d(eq_8, name="magnetic well", ax=ax, lw=2, linecolor="orange")
plt.grid()

In [ ]:
data_mesh = eq.compute(
    [
        "theta",
        "X",
        "Y",
        "Z",
        "|B|",
        "B",
        "p",
        "J",
        "e^theta",
        "e^rho",
        "e^zeta",
        "grad(alpha)",
    ],
    grid=Grid(nodes=coords_geo, sort=False),
)

### axial pressure profile
mu0 = 4 * np.pi * 1e-7

# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_axis = data_axis["p"]


# ---- Grid for midplane slice (zeta = pi, theta = 0, rho from 0 to 1) ----
rho_mid = np.linspace(0, 1, 100)
theta_mid = np.zeros_like(rho_mid)
zeta_mid = np.ones_like(rho_mid) * np.pi
grid_mid = Grid(np.stack([rho_mid, theta_mid, zeta_mid], axis=1))

# Compute on midplane profile
data_mid = eq.compute(["p", "|B|", "p"], grid=grid_mid)
B_mid = data_mid["|B|"]
p_mid = data_mid["p"]


X2d = np.reshape(data_mesh["X"], (Nz, Nt))
Y2d = np.reshape(data_mesh["Y"], (Nz, Nt))
Z2d = np.reshape(data_mesh["Z"], (Nz, Nt))
B2d = np.reshape(data_mesh["|B|"], (Nz, Nt))
J2d = np.reshape(data_mesh["J"], (Nz, Nt, 3))
theta2d = np.reshape(data_mesh["theta"], (Nz, Nt))

etheta = np.reshape(data_mesh["e^theta"], (Nz, Nt, 3))
ezeta = np.reshape(data_mesh["e^zeta"], (Nz, Nt, 3))
erho = np.reshape(data_mesh["e^rho"], (Nz, Nt, 3))

e_theta_n = etheta / np.linalg.norm(etheta, axis=-1)[:, :, None]
e_zeta_n = ezeta / np.linalg.norm(ezeta, axis=-1)[:, :, None]
e_rho_n = erho / np.linalg.norm(erho, axis=-1)[:, :, None]

J2d_n = J2d / np.linalg.norm(J2d, axis=-1)[:, :, None]

comp_theta = np.sum(J2d_n * e_theta_n, axis=-1)
comp_zeta = np.sum(J2d_n * e_zeta_n, axis=-1)
comp_rho = np.sum(J2d_n * e_rho_n, axis=-1)

Bv = np.reshape(data_mesh["B"], (Nz, Nt, 3))
B2d_n = Bv / np.linalg.norm(Bv, axis=-1)[:, :, None]

JdotB = np.sum(J2d_n * B2d_n, axis=-1)

# gradp = np.reshape(data_mesh['grad(p)'],(Nz,Nt,3))
# gradp_n =  gradp / np.linalg.norm(gradp,axis = -1)[:,:,None]

# Jdotgradp = np.sum(J2d_n*gradp_n,axis = -1)

# Bdotgradp = np.sum(B2d_n*gradp_n,axis = -1)

JdotB2d_n = np.sum(J2d_n * B2d_n, axis=-1)

gradalpha = np.reshape(data_mesh["grad(alpha)"], (Nz, Nt, 3))
gradalpha_n = gradalpha / np.linalg.norm(gradalpha, axis=-1)[:, :, None]

Jdotgradalpha_n = np.sum(J2d_n * gradalpha_n, axis=-1)

beta = 2 * (data_mesh["p"]) * mu_0 / (data_mesh["|B|"] ** 2 + 1e-20)
beta_axis = 2 * (data_axis["p"]) * mu_0 / (data_axis["|B|"] ** 2 + 1e-20)
beta_midplane = 2 * (data_mid["p"]) * mu_0 / (data_mid["|B|"] ** 2 + 1e-20)


beta2d = np.reshape(beta, (Nz, Nt))

fig, axs = plt.subplots(3, 3, figsize=(16, 11))

nlevels = 500
cmap = "coolwarm"

# field lines
i = 0
for line in lines[::10]:
    fieldlines = axs[0, 0].scatter(line[1], line[2], c=B_on_lines[i], s=0.2)
    i = i + 10
fig.colorbar(fieldlines, ax=axs[0, 0], label=r"|B|")

#

axs[0, 1].plot(zeta_axis, p_axis, label=r"$p$")
axs[0, 1].legend()

axs[0, 2].plot(rho_mid, p_mid, label=r"$p$")
axs[0, 2].legend()


im = axs[1, 0].contour(Y2d, theta2d, beta2d, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[1, 0])


im = axs[1, 1].plot(zeta_axis, beta_axis)
im = axs[1, 2].plot(rho_mid, beta_midplane)


# J
im = axs[2, 1].contour(Y2d, theta2d, comp_rho, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 1])

im = axs[2, 0].contour(Y2d, theta2d, Jdotgradalpha_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 0])

im = axs[2, 2].contour(Y2d, theta2d, JdotB2d_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 2])


# titles
axs[0, 0].set_title(r"$\rho = {}$ Field lines".format(rho_surf))
axs[0, 1].set_title("axial profile")
axs[0, 2].set_title("radial profile (midplane)")

axs[1, 0].set_title(r"$\beta$ at $\rho = {}$ ".format(rho_surf))
axs[1, 1].set_title(r"axial $\beta$")
axs[1, 2].set_title(r"radial $\beta$")

axs[2, 2].set_title(r"$J \cdot b$")
axs[2, 0].set_title(r"$J \cdot \nabla \alpha$")
axs[2, 1].set_title(r"$J \cdot e^\rho$")

axs[0, 0].set_ylabel(r"$Z$")
axs[0, 1].set_ylabel(r"$pressure(Pa)$")
axs[0, 2].set_ylabel(r"$pressure(Pa)$")

axs[2, 2].set_ylabel(r"$\theta$")
axs[2, 0].set_ylabel(r"$\theta$")
axs[2, 1].set_ylabel(r"$\theta$")


axs[2, 0].set_xlabel(r"$Y$")
axs[2, 1].set_xlabel(r"$Y$")
axs[2, 2].set_xlabel(r"$Y$")


axs[0, 0].grid()
axs[1, 1].grid()
axs[1, 2].grid()

axs[2, 2].grid()
axs[2, 0].grid()
axs[2, 1].grid()


data_mesh = eq_8.compute(
    [
        "theta",
        "X",
        "Y",
        "Z",
        "|B|",
        "B",
        "p",
        "J",
        "e^theta",
        "e^rho",
        "e^zeta",
        "grad(alpha)",
    ],
    grid=Grid(nodes=coords_geo, sort=False),
)

### axial pressure profile
mu0 = 4 * np.pi * 1e-7

# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq_8.compute(["|B|", "p"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_axis = data_axis["p"]


# ---- Grid for midplane slice (zeta = pi, theta = 0, rho from 0 to 1) ----
rho_mid = np.linspace(0, 1, 100)
theta_mid = np.zeros_like(rho_mid)
zeta_mid = np.ones_like(rho_mid) * np.pi
grid_mid = Grid(np.stack([rho_mid, theta_mid, zeta_mid], axis=1))

# Compute on midplane profile
data_mid = eq_8.compute(["p", "|B|", "p"], grid=grid_mid)
B_mid = data_mid["|B|"]
p_mid = data_mid["p"]


X2d = np.reshape(data_mesh["X"], (Nz, Nt))
Y2d = np.reshape(data_mesh["Y"], (Nz, Nt))
Z2d = np.reshape(data_mesh["Z"], (Nz, Nt))
B2d = np.reshape(data_mesh["|B|"], (Nz, Nt))
J2d = np.reshape(data_mesh["J"], (Nz, Nt, 3))
theta2d = np.reshape(data_mesh["theta"], (Nz, Nt))

etheta = np.reshape(data_mesh["e^theta"], (Nz, Nt, 3))
ezeta = np.reshape(data_mesh["e^zeta"], (Nz, Nt, 3))
erho = np.reshape(data_mesh["e^rho"], (Nz, Nt, 3))

e_theta_n = etheta / np.linalg.norm(etheta, axis=-1)[:, :, None]
e_zeta_n = ezeta / np.linalg.norm(ezeta, axis=-1)[:, :, None]
e_rho_n = erho / np.linalg.norm(erho, axis=-1)[:, :, None]

J2d_n = J2d / np.linalg.norm(J2d, axis=-1)[:, :, None]

comp_theta = np.sum(J2d_n * e_theta_n, axis=-1)
comp_zeta = np.sum(J2d_n * e_zeta_n, axis=-1)
comp_rho = np.sum(J2d_n * e_rho_n, axis=-1)

Bv = np.reshape(data_mesh["B"], (Nz, Nt, 3))
B2d_n = Bv / np.linalg.norm(Bv, axis=-1)[:, :, None]

JdotB = np.sum(J2d_n * B2d_n, axis=-1)

# gradp = np.reshape(data_mesh['grad(p)'],(Nz,Nt,3))
# gradp_n =  gradp / np.linalg.norm(gradp,axis = -1)[:,:,None]

# Jdotgradp = np.sum(J2d_n*gradp_n,axis = -1)

# Bdotgradp = np.sum(B2d_n*gradp_n,axis = -1)

JdotB2d_n = np.sum(J2d_n * B2d_n, axis=-1)

gradalpha = np.reshape(data_mesh["grad(alpha)"], (Nz, Nt, 3))
gradalpha_n = gradalpha / np.linalg.norm(gradalpha, axis=-1)[:, :, None]

Jdotgradalpha_n = np.sum(J2d_n * gradalpha_n, axis=-1)

beta = 2 * (data_mesh["p"]) * mu_0 / (data_mesh["|B|"] ** 2 + 1e-20)
beta_axis = 2 * (data_axis["p"]) * mu_0 / (data_axis["|B|"] ** 2 + 1e-20)
beta_midplane = 2 * (data_mid["p"]) * mu_0 / (data_mid["|B|"] ** 2 + 1e-20)


beta2d = np.reshape(beta, (Nz, Nt))


nlevels = 500
cmap = "coolwarm"

# field lines
i = 0
for line in lines[::10]:
    fieldlines = axs[0, 0].scatter(line[1], line[2], c=B_on_lines[i], s=0.2)
    i = i + 10
fig.colorbar(fieldlines, ax=axs[0, 0], label=r"|B|")

#

axs[0, 1].plot(zeta_axis, p_axis, label=r"$p$")
axs[0, 1].legend()

axs[0, 2].plot(rho_mid, p_mid, label=r"$p$")
axs[0, 2].legend()


im = axs[1, 0].contour(Y2d, theta2d, beta2d, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[1, 0])


im = axs[1, 1].plot(zeta_axis, beta_axis)
im = axs[1, 2].plot(rho_mid, beta_midplane)